In [3]:
#Import Packages

import xarray as xr
import matplotlib.pyplot as plt
import numpy as np 


In [4]:
#Load data from vortex directory
vortex_path = '/vortex/clidex/data/NEMO/VIKING20X/SSH/'
fname_prefix = '1_VIKING20X.L46-KFS003_1d_'
fname_suffix = '_45W_80W_30N_57N.nc'
ds_ssh = xr.open_mfdataset(vortex_path + fname_prefix + '*_zeromean' + fname_suffix)

#Cut off land mass
ds_ssh = ds_ssh.where(ds_ssh != 0.0)

#Throw out data and compromise to monthly data
#Create time series to chunk data for monthly data
time_series = np.arange(ds_ssh.time_counter[0].to_numpy(), ds_ssh.time_counter[-1].to_numpy(), np.timedelta64(1, 'M'),
                        dtype='datetime64[M]').astype('datetime64[D]')
ds_ssh = ds_ssh.groupby_bins('time_counter', time_series, right = False).mean(dim = 'time_counter')

#Format time series back to original form
time_series = np.delete(time_series, -1)
ds_ssh['time_counter_bins'] = time_series
ds_ssh.rename

#Throw out time dimension in lat/lon
ds_ssh['nav_lat'] = ds_ssh['nav_lat'][0]
ds_ssh['nav_lon'] = ds_ssh['nav_lon'][0]

ds_ssh = ds_ssh.rename({'time_counter_bins': 'time_counter'})

#Save file in new cdf file

ds_ssh.to_netcdf('/mnt/data/SSH/ssh_monthly.nc')

/home/milon.miah/miniconda3/lib/python3.9/site-packages/xarray/core/indexing.py:1227: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
